# 1 Importar librerías y cargar datos

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')


In [2]:
data = pd.read_csv("./data/data.csv", error_bad_lines=False)

b'Skipping line 2810: expected 2 fields, saw 5\nSkipping line 4641: expected 2 fields, saw 5\nSkipping line 7171: expected 2 fields, saw 5\nSkipping line 11220: expected 2 fields, saw 5\nSkipping line 13809: expected 2 fields, saw 5\nSkipping line 14132: expected 2 fields, saw 5\nSkipping line 14293: expected 2 fields, saw 5\nSkipping line 14865: expected 2 fields, saw 5\nSkipping line 17419: expected 2 fields, saw 5\nSkipping line 22801: expected 2 fields, saw 5\nSkipping line 25001: expected 2 fields, saw 5\nSkipping line 26603: expected 2 fields, saw 5\nSkipping line 26742: expected 2 fields, saw 5\nSkipping line 29702: expected 2 fields, saw 5\nSkipping line 32767: expected 2 fields, saw 5\nSkipping line 32878: expected 2 fields, saw 5\nSkipping line 35643: expected 2 fields, saw 5\nSkipping line 36550: expected 2 fields, saw 5\nSkipping line 38732: expected 2 fields, saw 5\nSkipping line 40567: expected 2 fields, saw 5\nSkipping line 40576: expected 2 fields, saw 5\nSkipping line 

In [3]:
data.head()

,password,strength
0,kzde5577,1
1,kino3434,1
2,visi7k1yr,1
3,megzy123,1
4,lamborghin1,1


In [4]:
data['strength'].unique()

array([1, 2, 0])

# 2 Comprobar los nulos

In [5]:
data.isna().sum()

password    1
strength    0
dtype: int64

In [6]:
data.shape

(669640, 2)

In [7]:
data.dropna(inplace = True)

# 3. Tenerlos datos en una tupla

In [8]:
import random

In [9]:
password_tuple = np.array(data)
password_tuple

array([['kzde5577', 1],
       ['kino3434', 1],
       ['visi7k1yr', 1],
       ...,
       ['184520socram', 1],
       ['marken22a', 1],
       ['fxx4pw4g', 1]], dtype=object)

In [10]:
random.shuffle(password_tuple)

In [11]:
x = [labels[0] for labels in password_tuple]

In [12]:
y = [labels[1] for labels in password_tuple]

# 4. crear una función personalizada para dividir la entrada en caracteres de la lista

In [13]:
def word_divide_char(inputs):
    character=[]
    for i in inputs:
        character.append(i)
    return character

In [14]:
word_divide_char('kzde5577')

['k', 'z', 'd', 'e', '5', '5', '7', '7']

# 5. importar vectorizador TF-IDF para convertir datos de cadena en datos numéricos

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [16]:
vectorizer=TfidfVectorizer(tokenizer=word_divide_char)

In [17]:
X=vectorizer.fit_transform(x)

In [18]:
X.shape

(669639, 136)

In [19]:
#vectorizer.get_feature_names()

In [20]:
first_document_vector=X[0]
first_document_vector

<1x136 sparse matrix of type '<class 'numpy.float64'>'
	with 6 stored elements in Compressed Sparse Row format>

In [21]:
#first_document_vector.T.todense()

In [22]:
df=pd.DataFrame(first_document_vector.T.todense(),index=vectorizer.get_feature_names(),columns=['TF-IDF'])
df.sort_values(by=['TF-IDF'],ascending=False)

,TF-IDF
7,0.592051
5,0.566575
z,0.335685
k,0.291981
d,0.285509
...,...
?,0.000000
>,0.000000
=,0.000000
<,0.000000


# 6. Aplicar LogisticRegression

dividir datos en entrenar y probar

     entrenar ---> Para aprender la relación dentro de los datos,
     
     prueba -> Para hacer predicciones, y estos datos de prueba no serán vistos para mi modelo

In [26]:
from sklearn.linear_model import LogisticRegression

In [28]:
from sklearn.model_selection import train_test_split

In [29]:
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.2)

In [30]:
X_train.shape

(535711, 136)

In [39]:
clf=LogisticRegression(random_state=0,solver='liblinear')

In [40]:
clf.fit(X_train,y_train)

LogisticRegression(random_state=0, solver='liblinear')

#### Hacer prediccón

In [41]:
dt=np.array(['%@123abcd'])
pred=vectorizer.transform(dt)
clf.predict(pred)

array([2])

In [42]:
y_pred=clf.predict(X_test)
y_pred

array([2, 0, 1, ..., 2, 1, 1])

####  Comprobar la precisión usando confusion_matrix,accuracy_score

In [43]:
from sklearn.metrics import confusion_matrix,accuracy_score

In [44]:
cm=confusion_matrix(y_test,y_pred)
print(cm)
print(accuracy_score(y_test,y_pred))

[[ 3476 14561    30]
 [ 2234 94783  2221]
 [   70  5948 10605]]
0.8128546681799176


####  Crear un reporte para el modelo 

In [45]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.60      0.19      0.29     18067
           1       0.82      0.96      0.88     99238
           2       0.82      0.64      0.72     16623

    accuracy                           0.81    133928
   macro avg       0.75      0.60      0.63    133928
weighted avg       0.79      0.81      0.78    133928

